In [13]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import contractions
import nltk
from nltk.corpus import stopwords
import itertools
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
data=pd.read_csv("https://raw.githubusercontent.com/cipheraxat/Disaster-Response/master/data/disaster_messages.csv")
data.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [5]:
data["genre"].value_counts()

news      13068
direct    10782
social     2398
Name: genre, dtype: int64

In [6]:
data["id"].value_counts()

24779    3
7747     2
14246    2
25512    2
17553    2
        ..
27320    1
25273    1
29371    1
17085    1
2049     1
Name: id, Length: 26180, dtype: int64

In [ ]:
high_priority=[]

In [7]:
messages=data["message"]

In [8]:
messages[:5]

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [12]:
a=messages[0]
a

'Weather update - a cold front from Cuba that could pass over Haiti'

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tarushi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
def clean_sentences(sent_list):
    for i in range(0,len(sent_list)):
        a=sent_list[i]
        a=a.lower()
        a = ' '.join([word for word in a.split() if not word in set(stopwords.words('english'))])
        a =  re.sub('[^a-zA-Z]', ' ', a)
        a = ' '.join(a.split())
        sent_list[i]=a
    return sent_list

        

In [24]:
clean_sentences(messages[:500])

0        weather update cold front cuba could pass haiti
1                                              hurricane
2                                   looking someone name
3      un reports leogane destroyed hospital st croix...
4        says west side haiti rest country today tonight
                             ...                        
495    received message hope want know get answer go ...
496    cyber cafe carefour wayne working sent rep s b...
497    many us mangonese street christ roi pap need f...
498    urjent need need food pver people families wit...
499                one manager distribuition his her car
Name: message, Length: 500, dtype: object

In [19]:
import spacy
from spacy import displacy
from collections import Counter
!python3 -m spacy download en
import en_core_web_sm
nlp = en_core_web_sm.load()

     |████████████████████████████████| 12.0 MB 284 kB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047105 sha256=5a68e92f9c7bf66c9b6c5742420402a46bce86fa60439829e47b9626ef25d9c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-decy5tdl/wheels/ee/4d/f7/563214122be1540b5f9197b52cb3ddb9c4a8070808b22d5a84
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/tarushi/.local/lib/python3.8/site-packages/en_core_web_sm -->
/home/tarushi/.local/lib/python3.8/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [31]:
high_emergency="this is an emergency extreme weather danger abandoned fire floods urgent "
doc=nlp(high_emergency)
x=doc.similarity(nlp(messages[1]))
x

<ipython-input-31-2254b796345d>:3: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  x=doc.similarity(nlp(messages[1]))


0.3938822633711599

In [36]:
priority_level=[]
for i in messages[:20]:
    x=doc.similarity(nlp(i))
    if(x>70):
        priority_level.append(1)
    elif(x<=70 and x>50):
        priority_level.append(2)
    elif(x<=50):
        priority_level.append(3)
        

<ipython-input-36-57737bee5b9b>:3: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  x=doc.similarity(nlp(i))


In [37]:
print(messages[:20])
print(priority_level)

0       weather update cold front cuba could pass haiti
1                                             hurricane
2                                  looking someone name
3     un reports leogane destroyed hospital st croix...
4       says west side haiti rest country today tonight
5                           information national palace
6                              storm sacred heart jesus
7                please need tents water silo thank you
8                     would like receive messages thank
9     croix des bouquets health issues workers santo...
10           there s nothing eat water starving thirsty
11               petionville need information regarding
12    thomassin number area named pyron would like w...
13           let s together need food delma didine area
14         information number order participate see use
15    comitee delmas rue street janvier impasse char...
16    need food water klecin dying hunger impasse ch...
17                     going call want call ou l